In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


img = cv2.imread('/content/vanishing point.PNG')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=10, maxLineGap=10)


In [ ]:

# Estimate vanishing point using RANSAC
def estimate_vanishing_point(lines):
    best_model = None
    best_score = 0
    for _ in range(100):
        # Sample two random lines
        idx1, idx2 = np.random.choice(len(lines), 2, replace=False)
        line1, line2 = lines[idx1][0], lines[idx2][0]

        # Calculate intersection point
        x1, y1, x2, y2 = line1
        x3, y3, x4, y4 = line2
        denom = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
        if denom == 0:
            continue
        x = ((x1*y2-y1*x2)*(x3-x4) - (x1-x2)*(x3*y4-y3*x4)) / denom
        y = ((x1*y2-y1*x2)*(y3-y4) - (y1-y2)*(x3*y4-y3*x4)) / denom

        # Count inliers
        score = 0
        for line in lines:
            x1, y1, x2, y2 = line[0]
            dist = np.abs((y2-y1)*x - (x2-x1)*y + x2*y1 - y2*x1) / np.sqrt((y2-y1)**2 + (x2-x1)**2)
            if dist < 10:
                score += 1
        if score > best_score:
            best_score = score
            best_model = (x, y)

    return best_model



In [ ]:
vanishing_point = estimate_vanishing_point(lines)

# Visualize the vanishing point
cv2.circle(img, (int(vanishing_point[0]), int(vanishing_point[1])), 5, (0, 0, 255), -1)
plt.imshow(img)

In [ ]:
vanishing_point